In [2]:
# created on April 25, 2022
# modified on April 25, 2022
# @author:          Jasmine Palaganas
# @email:           jpalag@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [3]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount("/gdrive")

Mounted at /gdrive


In [4]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [6]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "HoFDAbhMZAmUsJvlZAh9piN3v"
    consumer_secret = "HoTfp2chtiya4Fd1tuUDJ3o6plv2wppVDpONnpGh3DTmS3jFhb"
    access_token = "1516906141539651584-QtAdO3oMk5O5Bcyvf5GBS9U5YWQo9v"
    access_token_secret = "dBchehLBn4yhcip3x6vXXBY4PYC63M2ZY8OSgT4QgxO0G"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-122.383271, 47.502659, -122.286527, 47.732594]  # Seattle, Washington
    #             -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
    #             -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii  


    stream_listener = StreamListener(time_limit=500, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(track=["Elon Musk"], languages=['en'], encoding="utf-8")

{'id': 1518885630549319681, 'username': 'peroopeace', 'created_at': 'Tue Apr 26 09:32:08 +0000 2022', 'lng': 6.784844, 'lat': 6.1371965, 'text': 'Elon musk type of money go feet me sha @elonmusk @ABC  @Shibtoken'}
{'id': 1518885886213115906, 'username': 'AnkhMorporker', 'created_at': 'Tue Apr 26 09:33:09 +0000 2022', 'lng': -6.2476215, 'lat': 53.354902499999994, 'text': 'Elon Musk is a dick'}
{'id': 1518885976159965184, 'username': 'ChristopherG_W', 'created_at': 'Tue Apr 26 09:33:30 +0000 2022', 'lng': -0.102162, 'lat': 51.5879015, 'text': 'SCREW ELON MUSK. #RIPTwitter'}
{'id': 1518886014638444544, 'username': 'enigmatic_avi', 'created_at': 'Tue Apr 26 09:33:40 +0000 2022', 'lng': 72.874244, 'lat': 19.17414, 'text': "Nothing is so exhilarating than controlling the narrative.But it's refreshing to see Elon Musk behave like he sho… https://t.co/0I9u6OktGY"}
{'id': 1518886051749761025, 'username': 'Paul_Ensell', 'created_at': 'Tue Apr 26 09:33:48 +0000 2022', 'lng': -5.111084, 'lat': 50.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
